<a href="https://colab.research.google.com/github/farshid101/Thesis-2024/blob/main/Llama_3_1_8B_for_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://github.com/farshid101/Thesis-2024/blob/main/Final_clean%20pro%202.xlsx


import pandas as pd
df = pd.read_excel('/content/Final_clean pro 2.xlsx')
#ds = Dataset.from_pandas(df)
df

,Comments,Label
0,তোমরা সবাই এই ভাই তাকে একটু কোরোভিডিও গুলো দে...,positive
1,🎯 ব্রহ্মপুত্র নদের ওপর নির্মিত ৯.১৫ কিলোমিটারে...,positive
2,😊 ধন্যবাদ মাননীয় প্রধানমন্ত্রী শেখ হাসিনা,positive
3,2022 এর 25 এ জুন 🥰🥰,neutral
4,অনুভূতিটা এত সুন্দর ভাষায় প্রকাশ করতে পারতেছি...,positive
...,...,...
6370,সব পিকাব বন্দ করে দেয়া হোক,negative
6371,এটাতো গোটারি কথা কারণ কোরআনের বদলে গান বলে উদ্...,neutral
6372,এগুলা সব বিনপির এর কাজ,neutral
6373,এইবার ট্রাক বন্ধ করে দেয়া হোক,negative


In [ ]:
!pip install transformers datasets


In [ ]:
# # Mapping dictionary
# mapping = {'positive': 0, 'neutral': 1, 'negative': 2}

# # Replace values using map
# df['Label'] = df['Label'].map(mapping)

# df

In [ ]:
import pandas as pd

# Function to rename a single column
def change_column_name(df, old_name, new_name):
    df = df.rename(columns={old_name: new_name})
    return df

# Assuming df is your DataFrame
#df = change_column_name(df, old_name="Label", new_name="Label")
df = change_column_name(df, old_name="Comments", new_name="text")
df.head()  # Display the first few rows of the DataFrame
#text

,text,Label
0,তোমরা সবাই এই ভাই তাকে একটু কোরোভিডিও গুলো দে...,positive
1,🎯 ব্রহ্মপুত্র নদের ওপর নির্মিত ৯.১৫ কিলোমিটারে...,positive
2,😊 ধন্যবাদ মাননীয় প্রধানমন্ত্রী শেখ হাসিনা,positive
3,2022 এর 25 এ জুন 🥰🥰,neutral
4,অনুভূতিটা এত সুন্দর ভাষায় প্রকাশ করতে পারতেছি...,positive


In [ ]:
!pip install evaluate


In [ ]:
!pip install unsloth torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 31.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset
import evaluate
import numpy as np

# Assume `df` has already been processed with your mapping:
# Mapping dictionary
mapping = {'positive': 0, 'neutral': 1, 'negative': 2}

# Replace values using map
df['Label'] = df['Label'].map(mapping)

# Convert to Hugging Face Dataset format
data = {'text': df['text'].tolist(), 'label': df['Label'].tolist()}
dataset = Dataset.from_dict(data)
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Load the Model and Tokenizer
model_name = "unsloth/Qwen2-Math-7B-Instruct"  # Placeholder; replace with the actual Llama model identifier if available
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Tokenize the Dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Set Up Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs"
)

# Define a Compute Metrics Function
accuracy_metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    accuracy = accuracy_metric.compute(predictions=preds, references=labels)["accuracy"]
    return {"accuracy": accuracy}

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

# Train the Model
trainer.train()

# Make Predictions
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True)
    outputs = model(**inputs)
    prediction = np.argmax(outputs.logits.detach().numpy(), axis=1)[0]
    sentiment = ["positive", "neutral", "negative"][prediction]
    return sentiment

# Example prediction
text_to_predict = "অনুভূতিটা এত সুন্দর ভাষায় প্রকাশ করতে পারতেছি"
predicted_sentiment = predict_sentiment(text_to_predict)
print(f"Predicted Sentiment: {predicted_sentiment}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import torch
from unsloth import FastLanguageModel
from datasets import Dataset
import numpy as np
import pandas as pd

# Assume `df` is your DataFrame with 'text' and 'Label' columns
# Mapping dictionary
mapping = {'positive': 0, 'neutral': 1, 'negative': 2}
df['Label'] = df['Label'].map(mapping)

# Convert to Hugging Face Dataset format
data = {'text': df['text'].tolist(), 'label': df['Label'].tolist()}
dataset = Dataset.from_dict(data)
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Define parameters
max_seq_length = 2048  # Maximum sequence length
dtype = None  # Data type, None for auto-detection
load_in_4bit = True  # Use 4-bit quantization if desired

# Load the model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-bnb-4bit",  # Replace with desired model
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Tokenize the Dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Set up training arguments (assuming you're using Trainer class)
training_args = {
    "output_dir": "./results",
    "evaluation_strategy": "epoch",
    "learning_rate": 2e-5,
    "per_device_train_batch_size": 4,
    "num_train_epochs": 3,
    "weight_decay": 0.01,
    "logging_dir": "./logs"
}

# Define a compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    accuracy = (preds == labels).mean()  # Simple accuracy calculation
    return {"accuracy": accuracy}

# Create a Trainer (you might need to adjust this based on how FastLanguageModel works)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Train the Model
trainer.train()

# Make Predictions
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = np.argmax(outputs.logits.detach().numpy(), axis=1)[0]
    sentiment = ["positive", "neutral", "negative"][prediction]
    return sentiment

# Example prediction
text_to_predict = "The movie was fantastic!"
predicted_sentiment = predict_sentiment(text_to_predict)
print(f"Predicted Sentiment: {predicted_sentiment}")
